### Importing Required Libraries

In this cell, I import all the essential Python libraries needed for data preprocessing, image loading, and machine learning model development.  
- **TensorFlow/Keras** will be used to build and train the Convolutional Neural Network (CNN).  
- **ImageDataGenerator** helps load and augment images from folders.  
- **NumPy** is used for numerical operations.  
- **Matplotlib** is used later for visualising the training process.

These imports form the foundation for the rest of the project.


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import os

### Defining Dataset Paths

Here, I define the file paths to the training, validation, and testing datasets.  
The dataset is organised into separate folders for each split:

- `train` – used to train the CNN  
- `valid` – used to validate the model during training  
- `test` – used to evaluate final model performance  

Using folder-based datasets allows Keras’ `flow_from_directory` function to automatically assign labels based on folder names.


In [2]:
train_dir = "../data/AI-CA-Data/train"
valid_dir = "../data/AI-CA-Data/valid"
test_dir  = "../data/AI-CA-Data/test"

train_dir, valid_dir, test_dir


('../data/AI-CA-Data/train',
 '../data/AI-CA-Data/valid',
 '../data/AI-CA-Data/test')

### Loading the Training and Validation Data

In this cell, I load the images from the `train` and `valid` directories using Keras’ `flow_from_directory` function.

- Each subfolder inside the dataset represents a dog breed.
- `target_size` resizes all images to 224×224 pixels for input into the model.
- `batch_size` controls how many images are processed at once.
- `class_mode='categorical'` prepares the labels in one-hot encoded format for multi-class classification.

The output confirms how many images were loaded and that all 70 dog-breed classes were found.


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
batch_size = 32

# Training data generator (with augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

# Validation data generator (NO augmentation)
valid_datagen = ImageDataGenerator(
    rescale=1./255
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_data = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 7946 images belonging to 70 classes.
Found 700 images belonging to 70 classes.


### Building the CNN Model

In this cell, we build a Convolutional Neural Network (CNN) for dog breed classification.

- **Transfer Learning**: We use MobileNetV2, a pre-trained model on ImageNet, as the base to leverage existing knowledge.
- **Input Layer**: Takes 224×224×3 images as input.
- **Base Model**: MobileNetV2 with frozen weights (we'll use its learned features).
- **Custom Top Layers**: Add a Global Average Pooling layer to reduce dimensions, followed by a Dense layer with Dropout for regularization, and a final output layer with softmax for 70 dog breeds.
- **Compilation**: We use Adam optimizer, categorical cross-entropy loss, and accuracy as the metric.

This approach is efficient and performs well even with limited computational resources.

In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential

# Number of dog breed classes
num_classes = train_data.num_classes

# Load pre-trained MobileNetV2 (without top layers)
base_model = MobileNetV2(
    input_shape=(*img_size, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model weights
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 70)             │        17,990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,603,910 (9.93 MB)

 Trainable params: 345,926 (1.32 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

### Training the Model

In this cell, we train the compiled model on the training data.

- **Epochs**: Number of times the entire training dataset is passed through the network. We start with 15 epochs.
- **Steps per epoch**: Automatically calculated from the training data generator.
- **Validation data**: Used after each epoch to monitor model performance on unseen data.
- **History**: The training history object captures metrics (loss and accuracy) for both training and validation sets at each epoch.

This history will be used later to visualize how the model improved during training.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    filepath='../saved_models/best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# Training the model
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10,              # Safe for your GPU/CPU
    callbacks=[early_stop, checkpoint]
)

Epoch 1/10
 93/249 ━━━━━━━━━━━━━━━━━━━━ 2:29 959ms/step - accuracy: 0.2468 - loss: 3.2884